# K-Means For Disease Prevelance Across Four Different Age Brackets

#### Import necessary packages to read sql code and run kmeans on data

In [128]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [129]:
import psycopg2

#### Connect to Postgres database

In [130]:
conn = psycopg2.connect(
    host="localhost",
    database="mimic",
    user="postgres",
    password="ri123hanam!", 
    port = "5431")
cursor = conn.cursor()

#### This first SQL code goes through and identifies the specific disease identified as an elixhauser code 
#### A exlixhauser code is a predetermined integer code that associates with icd9 codes and groups together a disease into a specific category. Shown in the code below with the first case statement those icd9 codes are grouped to form Congestive Heart Failure - a disease group utilized in our analysis 
#### This code creates all the different groups we classify as a homogenous disease. Throughout our project we are trying to find if different groups can create subgroups and allow for diagnosis with multiple diseases rather than one.

In [131]:
sql = """
DROP TABLE IF EXISTS ELIXHAUSER_QUAN CASCADE;
CREATE TABLE ELIXHAUSER_QUAN AS
with icd as
(
  select hadm_id, seq_num, icd9_code
  from mimiciii.diagnoses_icd
  where seq_num != 1 -- we do not include the primary icd-9 code
)
, eliflg as
(
select hadm_id, seq_num, icd9_code
, CASE
  when icd9_code in ('39891','40201','40211','40291','40401','40403','40411','40413','40491','40493') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('4254','4255','4257','4258','4259') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('428') then 1
  else 0 end as CHF       /* Congestive heart failure */

, CASE
  when icd9_code in ('42613','42610','42612','99601','99604') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('4260','4267','4269','4270','4271','4272','4273','4274',
  '4276','4278','4279','7850','V450','V533') then 1
  else 0 end as ARRHY

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('0932','7463','7464','7465','7466','V422','V433') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('394','395','396','397','424') then 1
  else 0 end as VALVE     /* Valvular disease */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('4150','4151','4170','4178','4179') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('416') then 1
  else 0 end as PULMCIRC  /* Pulmonary circulation disorder */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('0930','4373','4431','4432','4438','4439','4471','5571','5579','V434') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('440','441') then 1
  else 0 end as PERIVASC  /* Peripheral vascular disorder */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 3) in ('401') then 1
  else 0 end as HTN       /* Hypertension, uncomplicated */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 3) in ('402','403','404','405') then 1
  else 0 end as HTNCX     /* Hypertension, complicated */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('3341','3440','3441','3442','3443','3444','3445','3446','3449') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('342','343') then 1
  else 0 end as PARA      /* Paralysis */

, CASE
  when icd9_code in ('33392') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('3319','3320','3321','3334','3335','3362','3481','3483','7803','7843') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('334','335','340','341','345') then 1
  else 0 end as NEURO     /* Other neurological */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('4168','4169','5064','5081','5088') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('490','491','492','493','494','495','496','500','501','502','503','504','505') then 1
  else 0 end as CHRNLUNG  /* Chronic pulmonary disease */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2500','2501','2502','2503') then 1
  else 0 end as DM        /* Diabetes w/o chronic complications*/

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2504','2505','2506','2507','2508','2509') then 1
  else 0 end as DMCX      /* Diabetes w/ chronic complications */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2409','2461','2468') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('243','244') then 1
  else 0 end as HYPOTHY   /* Hypothyroidism */

, CASE
  when icd9_code in ('40301','40311','40391','40402','40403','40412','40413','40492','40493') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('5880','V420','V451') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('585','586','V56') then 1
  else 0 end as RENLFAIL  /* Renal failure */

, CASE
  when icd9_code in ('07022','07023','07032','07033','07044','07054') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('0706','0709','4560','4561','4562','5722','5723','5724','5728',
    '5733','5734','5738','5739','V427') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('570','571') then 1
  else 0 end as LIVER     /* Liver disease */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('5317','5319','5327','5329','5337','5339','5347','5349') then 1
  else 0 end as ULCER     /* Chronic Peptic ulcer disease (includes bleeding only if obstruction is also present) */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 3) in ('042','043','044') then 1
  else 0 end as AIDS      /* HIV and AIDS */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2030','2386') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('200','201','202') then 1
  else 0 end as LYMPH     /* Lymphoma */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 3) in ('196','197','198','199') then 1
  else 0 end as METS      /* Metastatic cancer */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 3) in
  (
     '140','141','142','143','144','145','146','147','148','149','150','151','152'
    ,'153','154','155','156','157','158','159','160','161','162','163','164','165'
    ,'166','167','168','169','170','171','172','174','175','176','177','178','179'
    ,'180','181','182','183','184','185','186','187','188','189','190','191','192'
    ,'193','194','195'
  ) then 1
  else 0 end as TUMOR     /* Solid tumor without metastasis */

, CASE
  when icd9_code in ('72889','72930') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('7010','7100','7101','7102','7103','7104','7108','7109','7112','7193','7285') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('446','714','720','725') then 1
  else 0 end as ARTH              /* Rheumatoid arthritis/collagen vascular diseases */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2871','2873','2874','2875') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('286') then 1
  else 0 end as COAG      /* Coagulation deficiency */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2780') then 1
  else 0 end as OBESE     /* Obesity      */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('7832','7994') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('260','261','262','263') then 1
  else 0 end as WGHTLOSS  /* Weight loss */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2536') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('276') then 1
  else 0 end as LYTES     /* Fluid and electrolyte disorders */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2800') then 1
  else 0 end as BLDLOSS   /* Blood loss anemia */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2801','2808','2809') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('281') then 1
  else 0 end as ANEMDEF  /* Deficiency anemias */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2652','2911','2912','2913','2915','2918','2919','3030',
    '3039','3050','3575','4255','5353','5710','5711','5712','5713','V113') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('980') then 1
  else 0 end as ALCOHOL /* Alcohol abuse */

, CASE
  when icd9_code in ('V6542') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('3052','3053','3054','3055','3056','3057','3058','3059') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('292','304') then 1
  else 0 end as DRUG /* Drug abuse */

, CASE
  when icd9_code in ('29604','29614','29644','29654') then 1
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2938') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('295','297','298') then 1
  else 0 end as PSYCH /* Psychoses */

, CASE
  when SUBSTRING(icd9_code FROM 1 for 4) in ('2962','2963','2965','3004') then 1
  when SUBSTRING(icd9_code FROM 1 for 3) in ('309','311') then 1
  else 0 end as DEPRESS  /* Depression */
from icd
)
-- collapse the icd9_code specific flags into hadm_id specific flags
-- this groups comorbidities together for a single patient admission
, eligrp as
(
  select 
	hadm_id, max(chf) as chf, max(arrhy) as arrhy, max(valve) as valve, max(pulmcirc) as pulmcirc, 
  max(perivasc) as perivasc, max(htn) as htn, max(htncx) as htncx, max(para) as para, max(neuro) as neuro, 
  max(chrnlung) as chrnlung, max(dm) as dm, max(dmcx) as dmcx, max(hypothy) as hypothy, max(renlfail) as renlfail, 
  max(liver) as liver, max(ulcer) as ulcer, max(aids) as aids, max(lymph) as lymph, max(mets) as mets, max(tumor) as tumor, 
  max(arth) as arth, max(coag) as coag, max(obese) as obese, max(wghtloss) as wghtloss, max(lytes) as lytes, 
  max(bldloss) as bldloss, max(anemdef) as anemdef, max(alcohol) as alcohol, max(drug) as drug, max(psych) as psych, max(depress) as depress
from eliflg
group by hadm_id
)
-- now merge these flags together to define elixhauser
-- most are straightforward.. but hypertension flags are a bit more complicated

select adm.hadm_id, chf as CONGESTIVE_HEART_FAILURE, arrhy as CARDIAC_ARRHYTHMIAS, valve as VALVULAR_DISEASE, 
pulmcirc as PULMONARY_CIRCULATION, perivasc as PERIPHERAL_VASCULAR
-- we combine "htn" and "htncx" into "HYPERTENSION"
, case
    when htn = 1 then 1
    when htncx = 1 then 1
  else 0 end as HYPERTENSION
, para as PARALYSIS, neuro as OTHER_NEUROLOGICAL, chrnlung as CHRONIC_PULMONARY
-- only the more severe comorbidity (complicated diabetes) is kept
, case
    when dmcx = 1 then 0
    when dm = 1 then 1
  else 0 end as DIABETES_UNCOMPLICATED
, dmcx as DIABETES_COMPLICATED, hypothy as HYPOTHYROIDISM, renlfail as RENAL_FAILURE, liver as LIVER_DISEASE, ulcer as PEPTIC_ULCER, 
aids as AIDS, lymph as LYMPHOMA, mets as METASTATIC_CANCER
-- only the more severe comorbidity (metastatic cancer) is kept
, case
    when mets = 1 then 0
    when tumor = 1 then 1
  else 0 end as SOLID_TUMOR
, arth as RHEUMATOID_ARTHRITIS, coag as COAGULOPATHY, obese as OBESITY, wghtloss as WEIGHT_LOSS, lytes as FLUID_ELECTROLYTE, 
bldloss as BLOOD_LOSS_ANEMIA, anemdef as DEFICIENCY_ANEMIAS, alcohol as ALCOHOL_ABUSE, drug as DRUG_ABUSE, psych as PSYCHOSES
, depress as DEPRESSION

from mimiciii.admissions adm
left join eligrp eli
  on adm.hadm_id = eli.hadm_id
order by adm.hadm_id;

DROP TABLE IF EXISTS ELIXHAUSER_SUM;
CREATE TABLE ELIXHAUSER_SUM AS
(SELECT hadm_id,
       COALESCE(CONGESTIVE_HEART_FAILURE, 0) +
       COALESCE(CARDIAC_ARRHYTHMIAS, 0) +
       COALESCE(VALVULAR_DISEASE, 0) +
       COALESCE(PULMONARY_CIRCULATION, 0) +
       COALESCE(PERIPHERAL_VASCULAR, 0) +
       COALESCE(HYPERTENSION, 0) +
       COALESCE(PARALYSIS, 0) +
       COALESCE(OTHER_NEUROLOGICAL, 0) +
       COALESCE(CHRONIC_PULMONARY, 0) +
       COALESCE(DIABETES_UNCOMPLICATED, 0) +
       COALESCE(DIABETES_COMPLICATED, 0) +
       COALESCE(HYPOTHYROIDISM, 0) +
       COALESCE(RENAL_FAILURE, 0) +
       COALESCE(LIVER_DISEASE, 0) +
       COALESCE(PEPTIC_ULCER, 0) +
       COALESCE(AIDS, 0) +
       COALESCE(LYMPHOMA, 0) +
       COALESCE(SOLID_TUMOR, 0) +
       COALESCE(RHEUMATOID_ARTHRITIS, 0) +
       COALESCE(COAGULOPATHY, 0) +
       COALESCE(OBESITY, 0) +
       COALESCE(WEIGHT_LOSS, 0) +
       COALESCE(FLUID_ELECTROLYTE, 0) +
       COALESCE(BLOOD_LOSS_ANEMIA, 0) +
       COALESCE(DEFICIENCY_ANEMIAS, 0) +
       COALESCE(ALCOHOL_ABUSE, 0) +
       COALESCE(DRUG_ABUSE, 0) +
       COALESCE(PSYCHOSES, 0) +
       COALESCE(DEPRESSION, 0) AS Elixhauser_Score
FROM ELIXHAUSER_QUAN);
-- DELETE FROM ELIXHAUSER_SUM
-- WHERE Elixhauser_Score = 0;
SELECT * FROM ELIXHAUSER_SUM;
"""

#### This table now creates a patient info database so that we can look at factors about the patient in assessing why they have the disease and better group patients together based on it. 
#### The age extraction due to HIPAA laws was difficult to create but the code below shows how our group created the code to correctly identify a patients age. 

In [132]:
sql_patient = """
DROP TABLE IF EXISTS PATIENT_INFO;
CREATE TABLE PATIENT_INFO AS(
WITH admissions_with_age AS(
  SELECT
    a.subject_id,
    a.admittime,
    p.dob,
	a.hadm_id,
	a.admission_type,
    ROW_NUMBER() OVER (PARTITION BY a.subject_id ORDER BY a.admittime ASC) AS admission_rank,
    CASE 
      WHEN EXTRACT(YEAR FROM AGE(a.ADMITTIME, p.DOB)) < 16 THEN NULL
      WHEN EXTRACT(YEAR FROM AGE(a.ADMITTIME, p.DOB)) >= 300 THEN 90
      ELSE EXTRACT(YEAR FROM AGE(a.ADMITTIME, p.DOB))
    END AS age
  FROM
    mimiciii.admissions a
    JOIN mimiciii.patients p ON a.subject_id = p.subject_id
	WHERE a.admission_type = 'EMERGENCY' or a.admission_type = 'ELECTIVE'
)
SELECT
  subject_id,
  admission_type,
  hadm_id,
--   admittime,
--   dob,
  age
FROM
  admissions_with_age
WHERE
  admission_rank = 1
  AND age IS NOT NULL);

SELECT * FROM PATIENT_INFO;
"""

#### This sql code create the elixhauser tables that we utilize and keep track of for each individual patient to do analysis on 

In [133]:
sql_2 = """DROP TABLE IF EXISTS ELIXHAUSER_FINAL;
CREATE TABLE ELIXHAUSER_FINAL AS
(SELECT CONGESTIVE_HEART_FAILURE,
  CARDIAC_ARRHYTHMIAS,
  VALVULAR_DISEASE,
  PULMONARY_CIRCULATION,
  PERIPHERAL_VASCULAR,
  HYPERTENSION,
  PARALYSIS,
  OTHER_NEUROLOGICAL,
  CHRONIC_PULMONARY,
  DIABETES_UNCOMPLICATED,
  DIABETES_COMPLICATED,
  HYPOTHYROIDISM,
  RENAL_FAILURE,
  LIVER_DISEASE,
  PEPTIC_ULCER,
  AIDS,
  LYMPHOMA,
  SOLID_TUMOR,
  RHEUMATOID_ARTHRITIS,
  COAGULOPATHY,
  OBESITY,
  WEIGHT_LOSS,
  FLUID_ELECTROLYTE,
  BLOOD_LOSS_ANEMIA,
  DEFICIENCY_ANEMIAS,
  ALCOHOL_ABUSE,
  DRUG_ABUSE,
  PSYCHOSES,
  DEPRESSION, q.hadm_id
FROM ELIXHAUSER_QUAN as q
INNER JOIN ELIXHAUSER_SUM as s ON q.hadm_id = s.hadm_id);

SELECT * FROM ELIXHAUSER_FINAL;
"""

#### This sql code creates age brackets for different age subgroups and also breaks down a patient as an emergency or an elective -- this is useful in creating subgroups to perform different cllustering on and for LCA in another notebook 

In [134]:
sql_4 = """
SELECT *,
 CASE
    WHEN age BETWEEN 16 AND 24 THEN 1
    WHEN age BETWEEN 25 AND 44 THEN 2
    WHEN age BETWEEN 45 AND 64 THEN 3 -- Fixed the typo here, it should be '[45-64]' not '[45-66]'
    WHEN age BETWEEN 65 AND 84 THEN 4 -- Fixed the typo here, it was '5AND' instead of 'AND', and '[66-89]' instead of '[65-84]'
    ELSE 5 -- Adjusted the age bracket to start from 85
 END AS age_bracket,
 CASE
    WHEN admission_type = 'EMERGENCY' THEN 1
    ELSE 2 -- Adjusted the age bracket to start from 85
 END AS admission_type_encoded
FROM PATIENT_INFO as p
INNER JOIN ELIXHAUSER_FINAL as f ON f.hadm_id = p.hadm_id
WHERE f.cardiac_arrhythmias IS NOT NULL;
"""

In [138]:
elixhauser = pd.read_sql_query(sql, conn)

/var/folders/07/9pcptmds0fl8q4v1tqm04jph0000gn/T/ipykernel_1834/2469126667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  elixhauser = pd.read_sql_query(sql, conn)


In [139]:
create_elixhauser = pd.read_sql_query(sql_2, conn)

/var/folders/07/9pcptmds0fl8q4v1tqm04jph0000gn/T/ipykernel_1834/2060706808.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  create_elixhauser = pd.read_sql_query(sql_2, conn)


In [140]:
patients_create = pd.read_sql_query(sql_patient, conn)


/var/folders/07/9pcptmds0fl8q4v1tqm04jph0000gn/T/ipykernel_1834/2060651699.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patients_create = pd.read_sql_query(sql_patient, conn)


In [141]:
patients = pd.read_sql_query(sql_4, conn)

/var/folders/07/9pcptmds0fl8q4v1tqm04jph0000gn/T/ipykernel_1834/1306650811.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patients = pd.read_sql_query(sql_4, conn)


In [142]:
patients = patients.loc[:,~patients.columns.duplicated()].copy()

#### This merges the patient information table with the disease information table on the "hadm_id" which is the id for a singular hospital visit 

In [143]:
merged_table = patients.merge(elixhauser, left_on = 'hadm_id', right_on = 'hadm_id')

#### The age groups are then split up into different subgroups to do analysis on as age clearly affects health

In [144]:
age_group_1 = merged_table[(merged_table['age'] >= 16) &  (merged_table['age'] < 24)]
age_group_2 = merged_table[(merged_table['age'] >= 24) &  (merged_table['age'] < 44)]
age_group_3 = merged_table[(merged_table['age'] >= 44) &  (merged_table['age'] < 64)]
age_group_4 = merged_table[(merged_table['age'] >= 64) &  (merged_table['age'] < 84)]
age_group_5 = merged_table[(merged_table['age'] >= 84)]

#### The article that we are trying to replicate uses KMeans as a visualizaiton for the people trying to create results through LCA and Network Analysis. The K-means sets up specific code so that the researchers could look through and observe possible clustering based on patients different diseases. However there was no set way of doing this. Thus I have provided one example of how KMeans can be used in one age group. Using this code you can explore different clusters and which diseases are prevelant in that cluster. 

In [146]:
kmeans = KMeans()
kmeans.fit(age_group_1[age_group_1.columns[4:-3]])
clusters = kmeans.fit_predict(age_group_1[age_group_1.columns[4:-3]])

# dict to store clusters data
cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)

# prints the cluster dict
age_group_1['cluster'] = cluster_dict

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/var/folders/07/9pcptmds0fl8q4v1tqm04jph0000gn/T/ipykernel_1834/1890942393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_grou

#### Analysis on one cluster for the youngest subgroup 
##### There seems to be a large number of cardiac arrhythmias in this subgroup and fluid electrolyte. This subgroups is not discussed in our paper so there doesn't seem to be a clear subgroup that can be created in this cluster. Looking at LCA we can establish this for certain. 

In [147]:
age_group_1[age_group_1['cluster'] == 0].sum(axis = 0)

subject_id                                                            2933306
admission_type              EMERGENCYEMERGENCYEMERGENCYEMERGENCYEMERGENCYE...
hadm_id                                                              12325474
age                                                                    1636.0
congestive_heart_failure                                                    4
cardiac_arrhythmias                                                        79
valvular_disease                                                            4
pulmonary_circulation                                                       1
peripheral_vascular                                                         0
hypertension                                                                8
paralysis                                                                   1
other_neurological                                                          4
chronic_pulmonary                                               

#### After further looking at the paper and discussing the paper with other teams and how they were attempting the project. I realized that I did not implement the proportions correctly and need to fix it. However the code frame for this seems to be replicated from the paper. 